### Steam Store Crawling
- 가상환경에서 패키지 설치 후 진행

In [2]:
from bs4 import BeautifulSoup
import requests
from collections import deque
import time
import numpy as np
import json

In [3]:
# appid file path 새로 지정요망
file_path = r"C:\Users\NT550-052\OneDrive\바탕 화면\ASAC\프로젝트\ML 프로젝트\appid_2022.txt"
file = open(file_path,"r")

In [4]:
appids = []
while True:
    line = file.readline()
    if not line: break
    appids.append(int(line.strip()))
appids

[1794680,
 1332010,
 1583720,
 1290000,
 1593500,
 1533420,
 1999360,
 1687950,
 1942280,
 1684930,
 1817070,
 1424330,
 1868140,
 1985690,
 1677770,
 1455840,
 1167630,
 1059990,
 1260520,
 1901370,
 1586800,
 1948280,
 1969080,
 1061910,
 2070550,
 658920,
 1284190,
 1665190,
 1718570,
 799640,
 1927720,
 1328840,
 1293180,
 1275670,
 674140,
 2237980,
 1574270,
 1829980,
 1350650,
 1262350,
 1171320,
 2206340,
 1776970,
 2088570,
 1599600,
 1740100,
 2135690,
 1865780,
 1115050,
 1239690,
 1827680,
 1732180,
 1451940,
 1599020,
 1935660,
 1035990,
 1657630,
 1359980,
 975370,
 1269640,
 1336490,
 1902490,
 648800,
 1955330,
 1062520,
 1148760,
 2083040,
 2106520,
 615120,
 1313140,
 1535560,
 1848450,
 799600,
 1522870,
 2113430,
 1997680,
 1717770,
 1574580,
 1215390,
 2240530,
 1986840,
 2058180,
 1488200,
 2207540,
 949480,
 1562700,
 2085360,
 1903370,
 2124380,
 1718870,
 1888430,
 1939160,
 1515210,
 2057080,
 1817940,
 1629530,
 1834870,
 1706930,
 1703340,
 1401590,
 1725640

In [5]:
Z = ["required_age","is_free","controller_support","dlc","metacritic","platforms","recommendations","achievements","release_date"]

In [6]:
final_data = {}

In [7]:
def please(id,idx,res):
    g_info = {}
    try:
        data = res.json()
        if data[str(id)]["success"] == True: # success == true
            keys = data[str(id)]["data"].keys()
            main = data[str(id)]["data"]
            
            for elem in Z:
                if elem in keys:
                    if elem in ["recommendations","achievements"]:
                        g_info[elem] = main[elem]["total"]
                    elif elem == "release_date":
                        g_info[elem] = main[elem]["date"]
                    elif elem == "metacritic":
                        g_info[elem] = main[elem]["score"]
                    elif elem == "dlc":
                        g_info[elem] = len(main[elem])
                    else:
                        g_info[elem] = main[elem]
                else:
                    g_info[elem] = np.nan
            print(idx,id,g_info)
        else: # success == false
            for elem in Z:
                g_info[elem] = np.nan
            print("error",idx,id,g_info)
                
    except: # 빈 페이지 load되는 데이터 
        for elem in Z:
            g_info[elem] = np.nan
        print("error",idx,id,g_info)
            
    finally:
        final_data[str(id)] = g_info
        

In [8]:
for idx,id in enumerate(appids[:2]):
    path = f"https://store.steampowered.com/api/appdetails?appids={id}"

    res = requests.get(path)
    if res.status_code == 200:
        please(id,idx,res)
    else:
        time.sleep(60)
        please(id,idx,res)
    time.sleep(1.5)
print(final_data)
    

0 1794680 {'required_age': 0, 'is_free': False, 'controller_support': 'full', 'dlc': 3, 'metacritic': 87, 'platforms': {'windows': True, 'mac': True, 'linux': False}, 'recommendations': 186539, 'achievements': 184, 'release_date': '20 Oct, 2022'}
1 1332010 {'required_age': 0, 'is_free': False, 'controller_support': 'full', 'dlc': 1, 'metacritic': nan, 'platforms': {'windows': True, 'mac': False, 'linux': False}, 'recommendations': 105559, 'achievements': 24, 'release_date': '19 Jul, 2022'}
{'1794680': {'required_age': 0, 'is_free': False, 'controller_support': 'full', 'dlc': 3, 'metacritic': 87, 'platforms': {'windows': True, 'mac': True, 'linux': False}, 'recommendations': 186539, 'achievements': 184, 'release_date': '20 Oct, 2022'}, '1332010': {'required_age': 0, 'is_free': False, 'controller_support': 'full', 'dlc': 1, 'metacritic': nan, 'platforms': {'windows': True, 'mac': False, 'linux': False}, 'recommendations': 105559, 'achievements': 24, 'release_date': '19 Jul, 2022'}}


In [9]:
final_data 

{'1794680': {'required_age': 0,
  'is_free': False,
  'controller_support': 'full',
  'dlc': 3,
  'metacritic': 87,
  'platforms': {'windows': True, 'mac': True, 'linux': False},
  'recommendations': 186539,
  'achievements': 184,
  'release_date': '20 Oct, 2022'},
 '1332010': {'required_age': 0,
  'is_free': False,
  'controller_support': 'full',
  'dlc': 1,
  'metacritic': nan,
  'platforms': {'windows': True, 'mac': False, 'linux': False},
  'recommendations': 105559,
  'achievements': 24,
  'release_date': '19 Jul, 2022'}}

In [ ]:
# sava_path 지정 요망 
save_path = ".\steam_store_data.json"
with open(save_path,'w') as f:
  json.dump(final_data, f, ensure_ascii=False, indent=4)